In [1]:
import PyPDF2

0 read all lines from pdf

In [4]:
with open('tesla.pdf', 'rb') as pdf_file:
  pdf_reader = PyPDF2.PdfFileReader(pdf_file)
  extracted_text = ""
  for page_number in range(pdf_reader.numPages):
    page = pdf_reader.getPage(page_number)
    extracted_text += page.extract_text()
print(extracted_text)

Q1 2023 Update
1Highlights 03
Financial Summary 04
Operational Summary 06
Vehicle Capacity 07
Core Technology 08
Other Highlights 09
Outlook 10
Photos & Charts 11
Key Metrics 19
Financial Statements 22
Additional Information 28S U M M A R Y H I G H L I G H T S  
(1) Excludes SBC (stock -based compensation).
(2) Free cash flow = operating cash flow less capex.
(3) Includes cash, cash equivalents and investments.Profitability 11.4% operating margin in Q1
$2.7B GAAP operating income in Q1
$2.5B GAAP net income in Q1
$2.9B non -GAAP net income1in Q1In the current macroeconomic environment, we see this year as a unique 
opportunity for Tesla. As many carmakers are working through challenges with the 
unit economics of their EV programs, we aim to leverage our position as a cost 
leader. We are focused on rapidly growing production, investments in autonomy 
and vehicle software, and remaining on track with our growth investments.
Our near -term pricing strategy considers a long -term view on

In [6]:
with open('output_lines.txt', 'w', encoding='utf-8') as output_file:
  output_file.write(extracted_text)

1.tabula

In [7]:
import tabula

In [16]:
pdf_file = "tesla.pdf"
tables = tabula.read_pdf(pdf_file, pages="all")

output_dir = "tabula_output/"

for i, table_df in enumerate(tables):
  file_name = f"table_{i}.csv"
  output_file_path = output_dir + file_name
  table_df.to_csv(output_file_path, index=False)

is a little not organized

2.pdfplumber(not working very well)

In [10]:
# pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 9.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.6 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pdfplumber
import csv

exclude empty

In [9]:
def is_mostly_empty(table, threshold=0.8):
  total_cells = len(table) * len(table[0])
  empty_cells = sum(cell is None or cell == ""  for row in table for cell in row)
  empty_ratio = empty_cells / total_cells
  # print(empty_ratio)
  return empty_ratio >= threshold

In [25]:
pdf_file = "tesla.pdf"
pdf = pdfplumber.open(pdf_file)

tables = []
for page in pdf.pages:
  page_tables = page.extract_tables()
  tables.extend(page_tables)


output_dir = "pdfplumber_tables/"

for i, table in enumerate(tables):
  if is_mostly_empty(table) or len(table) < 2:
    continue
  
  print(table)
  csv_file_path = f"{output_dir}table_{i}.csv"

  with open(csv_file_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)

    for row in table:
      csv_writer.writerow(row)
      
pdf.close()

0.23076923076923078
[['($ in millions, except percentages and per share data)', 'Q1-2022', 'Q2-2022', 'Q3-2022', 'Q4-2022', 'Q1-2023', 'YoY'], ['Total automotive revenues', '16,861', '14,602', '18,692', '21,307', '19,963', '18%'], ['Energy generation and storage revenue', '616', '866', '1,117', '1,310', '1,529', '148%'], ['Services and other revenue', '1,279', '1,466', '1,645', '1,701', '1,837', '44%'], ['', '', '', '', '', '', ''], ['Total revenues', '18,756', '16,934', '21,454', '24,318', '23,329', '24%'], ['Total gross profit', '5,460', '4,234', '5,382', '5,777', '4,511', '-17%'], ['Total GAAP gross margin', '29.1%', '25.0%', '25.1%', '23.8%', '19.3%', '-977 bp'], ['', '', '', '', '', '', ''], ['Operating expenses', '1,857', '1,770', '1,694', '1,876', '1,847', '-1%'], ['Income from operations', '3,603', '2,464', '3,688', '3,901', '2,664', '-26%'], ['Operating margin', '19.2%', '14.6%', '17.2%', '16.0%', '11.4%', '-779 bp'], ['', '', '', '', '', '', ''], ['Adjusted EBITDA', '5,023', 

9/12

next: extract text+tables, if url, preserve url

one file

In [10]:
pdf_file = "tesla.pdf"
output_file = "output.md"

def extract_text_and_tables_to_md(pdf_path, md_file):
   with pdfplumber.open(pdf_path) as pdf:
       with open(md_file, 'w', encoding='utf-8') as md:
           for page_num, page in enumerate(pdf.pages, start=1):
               md.write(f'## Page {page_num}\n\n')
               md.write(page.extract_text())
               md.write('\n\n')

               tables = page.extract_tables()
               if tables:
                   md.write('### Tables\n\n')
                   for table_num, table in enumerate(tables, start=1):
                       if is_mostly_empty(table) or len(table) < 2:
                        continue
                       md.write(f'#### Table {table_num}\n\n')
                       for row in table:
                           if row:
                               md.write('|'.join(str(cell) for cell in row) + '\n')
                       md.write('\n')


extract_text_and_tables_to_md(pdf_file, output_file)


0.23076923076923078
0.24489795918367346
0.13636363636363635
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.7567567567567568
0.8577235772357723
0.8452380952380952
0.8571428571428571
0.08333333333333333
0.08333333333333333


python script
return text table
other report

own task to test

finacial report for knowledge base